# Data Exploration Notebook

## Objectives:
1. Load and inspect all data sources
2. Check data quality and completeness
3. Identify missing data patterns
4. Visualize data distributions
5. Document data issues and cleaning needs

## Data Sources:
- PV production data (solar panels, inverter, battery)
- Room temperature and humidity data
- Weather forecast and current weather data
- Heating relay states
- Energy consumption by category
- Energy prices
- Battery data (if different from PV)
- EV charging data (if available)

In [ ]:
# Import required libraries
import sys
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import asyncio
import warnings
warnings.filterwarnings('ignore')

# Add pems_v2 directory to path for imports
sys.path.append(str(Path('../pems_v2').resolve()))

# Import project modules
from analysis.core.data_extraction import DataExtractor
from config.settings import PEMSSettings

# Set up plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

## 1. Data Loading

Load data for analysis period (last 30 days by default)

In [ ]:
# Initialize settings and data extractor
settings = PEMSSettings()
extractor = DataExtractor(settings)

# Define analysis period
end_date = datetime.now()
start_date = end_date - timedelta(days=30)

print(f"Analysis period: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")

In [ ]:
# Extract all data sources
async def extract_all_data():
    """Extract all available data sources."""
    print("Extracting data...")
    
    # Extract each data type
    pv_data = await extractor.extract_pv_data(start_date, end_date)
    room_data = await extractor.extract_room_temperatures(start_date, end_date)
    weather_data = await extractor.extract_weather_data(start_date, end_date)
    current_weather = await extractor.extract_current_weather(start_date, end_date)
    relay_data = await extractor.extract_relay_states(start_date, end_date)
    consumption_data = await extractor.extract_energy_consumption(start_date, end_date)
    price_data = await extractor.extract_energy_prices(start_date, end_date)
    battery_data = await extractor.extract_battery_data(start_date, end_date)
    ev_data = await extractor.extract_ev_data(start_date, end_date)
    
    return {
        'pv': pv_data,
        'rooms': room_data,
        'weather': weather_data,
        'current_weather': current_weather,
        'relays': relay_data,
        'consumption': consumption_data,
        'prices': price_data,
        'battery': battery_data,
        'ev': ev_data
    }

# Run extraction
data = await extract_all_data()

## 2. Data Quality Assessment

Check each data source for completeness and quality issues

In [ ]:
# Function to generate data quality report
def generate_quality_report(df, name):
    """Generate a quality report for a dataframe."""
    if df is None or df.empty:
        return {
            'name': name,
            'status': 'EMPTY',
            'records': 0,
            'columns': [],
            'missing_pct': 100,
            'date_range': 'N/A'
        }
    
    return {
        'name': name,
        'status': 'OK',
        'records': len(df),
        'columns': list(df.columns),
        'missing_pct': round(df.isnull().sum().sum() / (len(df) * len(df.columns)) * 100, 2),
        'date_range': f"{df.index.min()} to {df.index.max()}" if len(df) > 0 else 'N/A'
    }

# Generate reports for all data sources
quality_reports = []

# Handle simple dataframes
for key, df in data.items():
    if key not in ['rooms', 'relays']:  # These are dictionaries
        quality_reports.append(generate_quality_report(df, key))

# Handle room data (dict of dataframes)
if isinstance(data['rooms'], dict):
    for room_name, room_df in data['rooms'].items():
        quality_reports.append(generate_quality_report(room_df, f'room_{room_name}'))

# Handle relay data (dict of dataframes)
if isinstance(data['relays'], dict):
    for room_name, relay_df in data['relays'].items():
        quality_reports.append(generate_quality_report(relay_df, f'relay_{room_name}'))

# Display quality report
quality_df = pd.DataFrame(quality_reports)
print("\nData Quality Summary:")
print("=" * 80)
display(quality_df)

## 3. Missing Data Analysis

Analyze patterns in missing data

In [ ]:
# Visualize missing data patterns for main datasets
fig, axes = plt.subplots(3, 2, figsize=(15, 12))
axes = axes.flatten()

datasets_to_plot = [
    ('PV Data', data['pv']),
    ('Weather Data', data['weather']),
    ('Current Weather', data['current_weather']),
    ('Consumption Data', data['consumption']),
    ('Price Data', data['prices']),
    ('Battery Data', data['battery'])
]

for idx, (name, df) in enumerate(datasets_to_plot):
    if df is not None and not df.empty and idx < len(axes):
        # Calculate missing data percentage per column
        missing_pct = (df.isnull().sum() / len(df) * 100).sort_values(ascending=False)
        
        # Plot only columns with some missing data
        missing_cols = missing_pct[missing_pct > 0]
        if len(missing_cols) > 0:
            missing_cols.plot(kind='bar', ax=axes[idx], color='coral')
            axes[idx].set_title(f'{name} - Missing Data by Column')
            axes[idx].set_ylabel('Missing %')
            axes[idx].set_xticklabels(axes[idx].get_xticklabels(), rotation=45, ha='right')
        else:
            axes[idx].text(0.5, 0.5, 'No missing data', 
                          ha='center', va='center', transform=axes[idx].transAxes)
            axes[idx].set_title(f'{name} - Missing Data by Column')
    else:
        if idx < len(axes):
            axes[idx].text(0.5, 0.5, f'{name}\nNo data available', 
                          ha='center', va='center', transform=axes[idx].transAxes)
            axes[idx].set_title(name)

plt.tight_layout()
plt.show()

In [ ]:
# Analyze time gaps in data
def analyze_time_gaps(df, name, threshold_minutes=60):
    """Analyze time gaps in the data."""
    if df is None or df.empty or len(df) < 2:
        return None
    
    # Calculate time differences
    time_diff = df.index.to_series().diff()
    
    # Find gaps larger than threshold
    large_gaps = time_diff[time_diff > pd.Timedelta(minutes=threshold_minutes)]
    
    if len(large_gaps) > 0:
        gap_info = {
            'dataset': name,
            'num_gaps': len(large_gaps),
            'max_gap_hours': large_gaps.max().total_seconds() / 3600,
            'avg_gap_hours': large_gaps.mean().total_seconds() / 3600,
            'gap_dates': large_gaps.index.tolist()[:5]  # First 5 gap dates
        }
        return gap_info
    return None

# Analyze gaps for all datasets
gap_analysis = []
for name, df in data.items():
    if isinstance(df, pd.DataFrame):
        gap_info = analyze_time_gaps(df, name)
        if gap_info:
            gap_analysis.append(gap_info)

if gap_analysis:
    gap_df = pd.DataFrame(gap_analysis)
    print("\nTime Gap Analysis (gaps > 1 hour):")
    print("=" * 80)
    display(gap_df[['dataset', 'num_gaps', 'max_gap_hours', 'avg_gap_hours']])
else:
    print("\nNo significant time gaps found in the data.")

## 4. Statistical Summaries

Generate statistical summaries for key variables

In [ ]:
# PV Data Statistics
if not data['pv'].empty:
    print("\nPV Production Statistics:")
    print("=" * 80)
    pv_stats = data['pv'][['InputPower', 'ACPowerToGrid', 'ACPowerToUser', 'SOC']].describe()
    display(pv_stats)
    
    # Calculate daily energy statistics
    if 'solar_energy_kwh' in data['pv'].columns:
        daily_energy = data['pv']['solar_energy_kwh'].resample('D').sum()
        print(f"\nDaily Solar Energy Production:")
        print(f"  Mean: {daily_energy.mean():.2f} kWh")
        print(f"  Max:  {daily_energy.max():.2f} kWh")
        print(f"  Min:  {daily_energy.min():.2f} kWh")

In [ ]:
# Room Temperature Statistics
if data['rooms']:
    print("\nRoom Temperature Statistics:")
    print("=" * 80)
    
    room_stats = []
    for room_name, room_df in data['rooms'].items():
        if not room_df.empty and 'temperature' in room_df.columns:
            stats = {
                'room': room_name,
                'mean_temp': room_df['temperature'].mean(),
                'min_temp': room_df['temperature'].min(),
                'max_temp': room_df['temperature'].max(),
                'std_temp': room_df['temperature'].std()
            }
            if 'humidity' in room_df.columns:
                stats['mean_humidity'] = room_df['humidity'].mean()
            room_stats.append(stats)
    
    if room_stats:
        room_stats_df = pd.DataFrame(room_stats).round(2)
        display(room_stats_df)

In [ ]:
# Energy Consumption Statistics
if not data['consumption'].empty:
    print("\nEnergy Consumption Statistics:")
    print("=" * 80)
    
    # Get power columns
    power_cols = [col for col in data['consumption'].columns if col.endswith('_power')]
    if power_cols:
        consumption_stats = data['consumption'][power_cols].describe()
        display(consumption_stats)
        
        # Calculate daily consumption by category
        print("\nDaily Energy Consumption by Category (kWh):")
        for col in power_cols:
            energy_col = col.replace('_power', '_energy_kwh')
            if energy_col in data['consumption'].columns:
                daily_consumption = data['consumption'][energy_col].resample('D').sum()
                print(f"  {col.replace('_power', '')}: {daily_consumption.mean():.2f} kWh/day")

## 5. Time Series Visualization

Visualize key time series to understand patterns

In [ ]:
# Plot PV production over time
if not data['pv'].empty:
    fig, axes = plt.subplots(3, 1, figsize=(15, 10), sharex=True)
    
    # PV Power
    if 'InputPower' in data['pv'].columns:
        data['pv']['InputPower'].resample('H').mean().plot(ax=axes[0], label='PV Input Power', color='orange')
        axes[0].set_ylabel('Power (W)')
        axes[0].set_title('Solar PV Production')
        axes[0].legend()
    
    # Battery State
    if 'SOC' in data['pv'].columns:
        data['pv']['SOC'].plot(ax=axes[1], label='Battery SOC', color='green')
        axes[1].set_ylabel('SOC (%)')
        axes[1].set_title('Battery State of Charge')
        axes[1].legend()
    
    # Power Flows
    flow_cols = ['ACPowerToGrid', 'ACPowerToUser']
    available_cols = [col for col in flow_cols if col in data['pv'].columns]
    if available_cols:
        data['pv'][available_cols].resample('H').mean().plot(ax=axes[2])
        axes[2].set_ylabel('Power (W)')
        axes[2].set_title('Power Flows')
        axes[2].legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Plot room temperatures
if data['rooms']:
    # Select up to 6 rooms with data
    rooms_with_data = [(name, df) for name, df in data['rooms'].items() 
                       if not df.empty and 'temperature' in df.columns][:6]
    
    if rooms_with_data:
        fig, axes = plt.subplots(2, 3, figsize=(15, 8), sharex=True)
        axes = axes.flatten()
        
        for idx, (room_name, room_df) in enumerate(rooms_with_data):
            if idx < len(axes):
                room_df['temperature'].resample('H').mean().plot(ax=axes[idx], label='Temperature')
                if 'target_temp' in room_df.columns:
                    room_df['target_temp'].resample('H').mean().plot(ax=axes[idx], 
                                                                     label='Target', 
                                                                     linestyle='--', 
                                                                     alpha=0.7)
                axes[idx].set_title(f'Room: {room_name}')
                axes[idx].set_ylabel('Temperature (°C)')
                axes[idx].legend()
        
        # Hide unused subplots
        for idx in range(len(rooms_with_data), len(axes)):
            axes[idx].set_visible(False)
        
        plt.tight_layout()
        plt.show()

## 6. Correlation Analysis

Analyze correlations between key variables

In [ ]:
# Create correlation matrix for PV and weather data
if not data['pv'].empty and not data['weather'].empty:
    # Merge PV and weather data
    merged_data = pd.merge(
        data['pv'][['InputPower', 'SOC', 'ACPowerToGrid', 'ACPowerToUser']].resample('H').mean(),
        data['weather'][['temperature_2m', 'shortwave_radiation', 'cloudcover']].resample('H').mean(),
        left_index=True, right_index=True, how='inner'
    )
    
    if not merged_data.empty:
        # Calculate correlation matrix
        corr_matrix = merged_data.corr()
        
        # Plot heatmap
        plt.figure(figsize=(10, 8))
        sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
                    square=True, linewidths=1, fmt='.2f')
        plt.title('Correlation Matrix: PV Production vs Weather')
        plt.tight_layout()
        plt.show()

## 7. Data Validation Results

Run comprehensive data validation

In [ ]:
# Run data validation
validation_results = extractor.validate_data_completeness(data)

print("\nData Validation Results:")
print("=" * 80)
print(f"Overall Status: {'COMPLETE' if validation_results['is_complete'] else 'INCOMPLETE'}")

if validation_results['missing_required']:
    print(f"\nMissing Required Data Sources:")
    for source in validation_results['missing_required']:
        print(f"  - {source}")

if validation_results['missing_optional']:
    print(f"\nMissing Optional Data Sources:")
    for source in validation_results['missing_optional']:
        print(f"  - {source}")

if validation_results['recommendations']:
    print(f"\nRecommendations:")
    for rec in validation_results['recommendations']:
        print(f"  - {rec}")

## 8. Summary and Recommendations

Based on the data exploration, provide actionable insights

In [ ]:
# Generate summary report
print("\nData Exploration Summary:")
print("=" * 80)

# Count available data sources
available_sources = sum(1 for key, value in data.items() 
                       if (isinstance(value, pd.DataFrame) and not value.empty) or 
                          (isinstance(value, dict) and len(value) > 0))

print(f"\nData Sources Available: {available_sources}/{len(data)}")

# Key findings
print("\nKey Findings:")

# PV System
if not data['pv'].empty:
    max_power = data['pv']['InputPower'].max() if 'InputPower' in data['pv'].columns else 0
    print(f"  - PV System: Maximum power recorded: {max_power:.0f}W")

# Rooms
if data['rooms']:
    print(f"  - Rooms: {len(data['rooms'])} rooms with temperature data")

# Heating
if data['relays']:
    print(f"  - Heating: {len(data['relays'])} rooms with relay data")

# EV Charging
if data['ev'].empty:
    print("  - EV Charging: No dedicated EV charging data (as expected)")

print("\nNext Steps:")
print("  1. Run pattern analysis on available data")
print("  2. Implement feature engineering for ML models")
print("  3. Create thermal models for rooms with sufficient data")
print("  4. Develop PV production forecasting features")

In [ ]:
# Save key statistics for later use
summary_stats = {
    'analysis_period': {'start': start_date, 'end': end_date},
    'data_sources': available_sources,
    'pv_max_power': data['pv']['InputPower'].max() if not data['pv'].empty and 'InputPower' in data['pv'].columns else None,
    'num_rooms': len(data['rooms']) if data['rooms'] else 0,
    'num_heating_zones': len(data['relays']) if data['relays'] else 0,
    'validation_complete': validation_results['is_complete']
}

print("\nAnalysis complete. Summary statistics saved.")